## Data import

In [2]:
import pandas as pd

In [3]:
# movies
movies = pd.read_csv('./data/rotten_tomatoes_movies.csv', index_col = 'rotten_tomatoes_link')

In [4]:
# rotten_tomatoes_critic_reviews was too big it had to be split to be uploaded to GITHub
filenames = ['rotten_tomatoes_critic_reviews.csv', 'rotten_tomato_reviews_1.csv', 'rotten_tomato_reviews_2.csv', 'rotten_tomato_reviews_3.csv', 'rotten_tomato_reviews_4.csv', 'rotten_tomato_reviews_5.csv']
reviews_list = []

for filename in filenames:
    review = pd.read_csv(f'./data/{filename}', index_col='rotten_tomatoes_link')
    reviews_list.append(review)

# combining the critics review dataset back together
reviews = pd.concat(reviews_list)

## Data preprocessing

### movies

In [5]:
# deleting useless columna movies_info, critics_consensus containing long strings
movies.drop('movie_info', axis=1, inplace=True)
movies.drop('critics_consensus', axis=1, inplace=True)

### reviews

In [6]:
# dropping rows with missing review_score values
reviews = reviews[reviews['review_score'].notna()]

# deleting useless column review_cotent containing long strings
reviews = reviews.drop('review_content', axis=1)

In [7]:
#unique values in review_score 
# unq = reviews['review_score'].unique()
# unq.sort()
# unq

#moje poznamky co napsat do zpravy
#FILTROVANI DAT NA ZAKLADE CRITICS_SCORE
# DLE RAUCHA JE TO OK..
# --- vime o tom a vysledek muze byt timto zpusobem zkreslen

In [8]:
# deleting reviews with non-numeric review scores (approx 80 000 values)
reviews = reviews[reviews['review_score'].str.contains('[A-Za-z]') == False]

# splitting review scores into columns
split = reviews['review_score'].str.split('/')

 #calculating percentage scores
scores =  pd.to_numeric(split.str.get(0)) / pd.to_numeric(split.str.get(1))
scores = scores*100

# assigning calculated values to the reviews scores column
reviews['review_score'] = scores

## Final dataset

In [9]:
#Merging the two datasets - reviews and movies
data = pd.merge(movies, reviews, how='inner', on = 'rotten_tomatoes_link')
data

,movie_title,content_rating,genres,directors,authors,actors,original_release_date,streaming_release_date,runtime,production_company,...,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count,critic_name,top_critic,publisher_name,review_type,review_score,review_date
rotten_tomatoes_link,,,,,,,,,,,,,,,,,,,,,
m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,...,254421.0,43,73,76,Ben McEachen,False,Sunday Mail (Australia),Fresh,70.0,2010-02-09
m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,...,254421.0,43,73,76,Nick Schager,False,Slant Magazine,Rotten,25.0,2010-02-10
m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,...,254421.0,43,73,76,Bill Goodykoontz,True,Arizona Republic,Fresh,70.0,2010-02-10
m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,...,254421.0,43,73,76,Jim Schembri,True,The Age (Australia),Fresh,60.0,2010-02-10
m/0814255,Percy Jackson & the Olympians: The Lightning T...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,2015-11-25,119.0,20th Century Fox,...,254421.0,43,73,76,Mark Adams,False,Daily Mirror (UK),Fresh,80.0,2010-02-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
m/zulu_dawn,Zulu Dawn,PG,"Action & Adventure, Art House & International,...",Douglas Hickox,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,2016-03-22,121.0,Tango Entertainment,...,4469.0,0,4,4,Brandon Judell,False,PopcornQ,Fresh,60.0,2005-08-14
m/zulu_dawn,Zulu Dawn,PG,"Action & Adventure, Art House & International,...",Douglas Hickox,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,2016-03-22,121.0,Tango Entertainment,...,4469.0,0,4,4,Cole Smithey,False,ColeSmithey.com,Fresh,80.0,2005-11-01
m/zulu_dawn,Zulu Dawn,PG,"Action & Adventure, Art House & International,...",Douglas Hickox,"Cy Endfield, Anthony Storey","Burt Lancaster, Peter O'Toole, Simon Ward, Joh...",1979-12-14,2016-03-22,121.0,Tango Entertainment,...,4469.0,0,4,4,Chuck O'Leary,False,Fantastica Daily,Rotten,40.0,2005-11-02


## Otazka 3

### New columns

In [10]:
# new column with review score intervals
def review_score_interval(row):
    if row['review_score'] >= 90:
        return '>90'
    elif row['review_score'] >= 70:
        return '70-90'
    elif row['review_score'] >= 50:
        return '50-70'
    elif row['review_score'] >= 30:
        return '30-50'
    else:
        return '<30'

data["review_score_interval"] = data.apply(review_score_interval, axis=1)

In [11]:
# new column with tomatometer rating interval
def tomatometer_rating_interval(row):
    if row['tomatometer_rating'] >= 90:
        return '>90'
    elif row['tomatometer_rating'] >= 70:
        return '70-90'
    elif row['tomatometer_rating'] >= 50:
        return '50-70'
    elif row['tomatometer_rating'] >= 30:
        return '30-50'
    else:
        return '<30'

data["tomatometer_rating_interval"] = data.apply(tomatometer_rating_interval, axis=1)

In [12]:
# new column with runtime intervals
def runtime_interval(row):
    if row['runtime'] >= 150:
        return '>2.5h'
    elif row['runtime'] >= 120:
        return '2-2.5'
    elif row['runtime'] >= 105:
        return '1.75-2h'
    elif row['runtime'] >= 90:
        return '1.5-1.75h'
    else:
        return '<1.5h'

data["runtime_interval"] = data.apply(runtime_interval, axis=1)

In [13]:
# new column with release interval
data = data[data['original_release_date'].notnull()]

def release(row):
    split = row["original_release_date"].split('-')
    year = split[0]
    return year
        
data["release_year"] = data.apply(release, axis=1)

def release_interval(row):
    year = row["release_year"]
    interval =  pd.to_numeric(year)//10*10
    return interval

data["release_interval"] = data.apply(release_interval, axis=1)

C:\Users\Dominika\AppData\Local\Temp\ipykernel_2460\766977739.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["release_year"] = data.apply(release, axis=1)
C:\Users\Dominika\AppData\Local\Temp\ipykernel_2460\766977739.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["release_interval"] = data.apply(release_interval, axis=1)


### 4ftMiner

In [14]:
from cleverminer import cleverminer

Cleverminer version  1.0.2  educational version. For production use (bugfixes and keeping compatibility), ask for PRO version.


In [16]:

df = data
df=df.loc[:, ('tomatometer_rating_interval', 'release_interval', 'production_company')]



clm = cleverminer(df=df,proc='4ftMiner',
               quantifiers= {'conf':0.8, 'Base': 50},
               ante ={
                    'attributes':[
                        {'name': 'production_company', 'type': 'subset', 'minlen': 1, 'maxlen': 1},
                        {'name': 'release_interval', 'type': 'subset', 'minlen': 1, 'maxlen': 1},
                    ], 'minlen':2, 'maxlen':2, 'type':'con'},
               succ ={
                    'attributes':[
                        {'name': 'tomatometer_rating_interval', 'type': 'one', 'value': '>90'},
                    ], 'minlen':1, 'maxlen':1, 'type':'con'}
               )

# print(clm.result)
clm.print_summary()
clm.print_rulelist()
clm.print_rule(2)

Cleverminer version 1.0.2. Note: This version is for personal and educational use only. If you need PRO version (support, fixing structures for compactibility in future versions for production deployment, additional development, licensing of commercial use of subroutines used), feel free to ask authors. Most of these functionalities are maintained in best-effort, as soon as this project is at given conditions for free use and rapid development is needed, they cannot be guaranteed.
Starting data preparation ...
Encoding columns into bit-form...
Encoding columns into bit-form...done
Data preparation finished.
Will go for  4ftMiner
Starting to mine rules.
Done. Total verifications : 415, rules 122,control number:0, times: prep 6.675905704498291, processing 8.33884072303772

CleverMiner task processing summary:

Task type : 4ftMiner
Number of verifications : 415
Number of rules : 122
Total time needed : 00h 00m 15s
Time of data preparation : 00h 00m 06s
Time of rule mining : 00h 00m 08s


